In [ ]:
# Find the start of every finger movement for every event - findthe minimum before
# def find_finger_events

# create the .dev file to install all packages for me!

# ordered by indices to the ecog. example output ordered by the same way.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 40 same lines - 25Hz
# each column is a finger
finger_data = pd.read_csv('finger_data.csv')
events_data = pd.read_csv('events_data.csv')
brain_data = pd.read_csv('brain_data.csv')
# turn to integers



# 200ms before
# 1ms current - the minimum
# 1000ms

# 5 graphs for each finger 1201ms long.







